In [2]:
### 세대구성원 수 + 소득으로 도시근로자 기준 월평균 소득 계산
import numpy as np

def determine_income_level(income_list, num_family):
  income_series = np.array([3482964, 
                          5415712,
                          7198649,
                          8248467,
                          8775071,
                          9563282,
                          10351493,
                          11139704])
  income_avg = int(round(np.array(income_list).sum(),0))
  print(income_avg)
  scaling_factors = [1.0,1.2,1.3,1.4,1.6]
  scaled_incomes = list(map(lambda factor: int(round(income_series[num_family - 1] * factor, 0)), scaling_factors))
  
  for i, standard in enumerate(scaled_incomes):
    if standard >= income_avg:
      income_level = i+1
      break
    elif scaled_incomes[-1] < income_avg:
      income_level = len(scaling_factors) + 1
      break
  return income_level



In [3]:
# 공통 질문 사항 함수화
def common_question():
  estate_status = input("부동산가액이 3억 3100만원 이하인가요?").strip()
  estate_map = {
  "예" : 0,
  "아니요" : 1
  }
  estate_status = estate_map.get(estate_status)

  num_family = int(input("세대구성원 수는 본인을 포함해 총 몇 명인가요?(태아 포함)")).strip() ## 세대구성원에 커서를 올렸을 때 상세 설명 필요
  income_family = input("세대구성원 중 본인과 배우자를 제외한 유소득자의 소득을 모두 입력해주세요.\n(예 : 342만원, 580만원, ...)").strip()
  income_tax_status = input("근로자 혹은 자영업자로서 5년 이상 소득세를 납부했나요?").strip()
  income_tax_map = {
  "예" : 0,
  "아니요" : 1
  }
  income_tax_status = income_tax_map.get(income_tax_status)

  return estate_status, num_family, income_family, income_tax_status

In [2]:
non_house = input("무주택세대구성원이신가요?").strip() ## 무주택세대구성원에 커서를 올렸을 때 상세 설명 필요
if non_house in ['네', '예', 'Y', 'Yes', '맞습니다']:
  marriage_status = input("신혼부부이신가요?(혼인 예정이거나 혼인신고일 기준 7년 이내)").strip()
  marriage_map = {
    "신혼부부" : 0,
    "모집공고일 기준 혼인신고일 7년 초과 기혼자" : 1,
    "미혼" : 2
  }
  marriage_status = marriage_map.get(marriage_status)
  
  regist_type = []
  if marriage_status != 2: # 기혼자(신혼부부 혹은 7년초과의 기혼자)
    child_status = input("출생일로부터 2년 이내인 자녀가 있으신가요?(임신 중인 태아 포함)").strip()
    child_map = {
      "신생아 양육" : 0,
      "2세 초과 미성년 자녀 양육" : 1,
      "자녀 없음" : 2
    }
    child_status = child_map.get(child_status)
    
    income_status = input("부부의 소득활동 형태가 어떻게 되시나요?").strip()
    income_map = {
      "맞벌이" : 0,
      "외벌이" : 1
    }
    income_status = income_map.get(income_status)
    
    if income_map == 0: ## 맞벌이
      user_income = int(input("작년 기준 본인의 월소득을 입력해주세요.(단위 : 만원)").strip())
      user_partner_income = int(input("작년 기준 배우자의 월소득을 입력해주세요.(단위 : 만원)").strip())

      estate_status, num_family, income_family, income_tax_status = common_question()

      income_list = [user_income] + [user_partner_income] + [int(txt[:-2]) for txt in income_family.split(", ")]
      income_level = determine_income_level(income_list, num_family)

      if marriage_status == 0: ## 신혼부부
        ## 소득 구분
        if income_level <= 2 and income_list[0] <= 3482964: 
          if child_status == 0:
            regist_type.append({
              "type" : "신혼부부 특별공급",
              "income_step" : (1,"신생아 우선공급")
            })
          elif child_status == 1:
            regist_type.append({
            "type" : "신혼부부 특별공급",
            "income_step" : (3,"우선공급")
            })

        elif income_level <= 5 and income_list[0] <= int(round(3482964*1.4, 0)):
          if child_status == 0:
            regist_type.append({
              "type" : "신혼부부 특별공급",
              "income_step" : (2,"신생아 일반공급")
            })
          elif child_status == 1:
            regist_type.append({
            "type" : "신혼부부 특별공급",
            "income_step" : (4,"일반공급")
            })
        elif income_level == 6 and estate_status == 0:
          regist_type.append({
              "type" : "신혼부부 특별공급",
              "income_step" : (5,"추첨공급")
            })
        else:
          pass
      elif marriage_status == 1: ## 7년 초과 부부(생애최초)

    elif income_map == 1: ## 외벌이
      user_income = int(input("작년 기준 본인 혹은 배우자의 월소득을 입력해주세요.(단위 : 만원)").strip())
      
      estate_status, num_family, income_family, income_tax_status = common_question()

      income_list = [user_income] + [int(txt[:-2]) for txt in input.split(", ")]
      income_level = determine_income_level(income_list, num_family)
      
      if income_level <= 1:
        if child_status == 0:
          regist_type.append({
            "type" : "신혼부부 특별공급",
            "income_step" : (1,"신생아 우선공급")
          })
        elif child_status == 1:
          regist_type.append({
          "type" : "신혼부부 특별공급",
          "income_step" : (3,"우선공급")
          })

      elif income_level <= 4:
        if child_status == 0:
          regist_type.append({
            "type" : "신혼부부 특별공급",
            "income_step" : (2,"신생아 일반공급")
          })
        elif child_status == 1:
          regist_type.append({
          "type" : "신혼부부 특별공급",
          "income_step" : (4,"일반공급")
          })
      elif income_level >= 5 and estate_status == 0:
        regist_type.append({
            "type" : "신혼부부 특별공급",
            "income_step" : (5,"추첨공급")
          })
      else:
        pass
  elif marriage_status == 2: ## 미혼
    user_income = int(input("작년 기준 본인의 월소득을 입력해주세요.(단위 : 만원)").strip())
      
    estate_status, num_family, income_family, income_tax_status = common_question()

    income_list = [user_income] + [int(txt[:-2]) for txt in input.split(", ")]
    income_level = determine_income_level(income_list, num_family)

    if num_family == 1:
      if income_level <= 5 and estate_status == 0: ## 1인 가구
        regist_type.append({
          "type" : "생애최초 특별공급",
          "income_step" : (5,"추첨공급")
        })
    
    if income_level < 3: ## 미혼모 가구 / 한부모 가정에 대한 생각...
      regist_type.append({
        "type" : "생애최초 특별공급",
        "income_step" : (3,"우선공급")
      })

    elif income_level <= 5:
      regist_type.append({
        "type" : "생애최초 특별공급",
        "income_step" : (4,"일반공급")
      })
      
    elif income_level == 6 and estate_status == 0:
      regist_type = {
          "type" : "생애최초 특별공급",
          "income_step" : (5,"추첨공급")
        }
    else:
      pass
    
else:
  print("무주택세대구성원이 아니시군요! 무순위 청약을 안내해드릴게요.")
  


"모집공고일 기준 5년 이상 소득세 납부 여부"

0


무주택자 / 신혼부부 : 신혼부부 | 7년초과 기혼 | 미혼 / 자녀 : 신생아 있음 | 미성년 자녀 있음 | 

In [ ]:
x = None
y = 231
z = [34, 12, 5]
l = [x] + [y] + z
l = [t for t in l if t != None]
l

[231, 34, 12, 5]

In [35]:
income_series = np.array([3482964, 
                          5415712,
                          7198649,
                          8248467,
                          8775071,
                          9563282,
                          10351493,
                          11139704])
scaling_factors = [1.0, 1.2, 1.3, 1.4, 1.6]
scaled_incomes = list(map(lambda factor: round(income_series[3-1] * factor, 0), scaling_factors))
income_avg = 9563282
  
for i, standard in enumerate(scaled_incomes):
  if standard >= income_avg:
    income_level = i+1
    print(f"소득 구분 {income_level}단계의 소득입니다.")
    break
  
  elif scaled_incomes[-1] < income_avg:
    income_level = len(scaling_factors)
    print(f"소득 구분 {income_level}단계의 소득입니다.")
    break
  

10078109.0
소득 구분 4단계의 소득입니다.


In [13]:
### 세대구성원 수 + 소득으로 도시근로자 기준 월평균 소득 계산
import numpy as np

def determine_category_by_income(income_list, num_family, 
                                 marriage_status = 0, income_status = 0, 
                                 child_status = 0):
  income_series = np.array([3482964, 
                          5415712,
                          7198649,
                          8248467,
                          8775071,
                          9563282,
                          10351493,
                          11139704])
  income_avg = int(round(np.array(income_list).mean(),0))
  scaling_factors = [1.0,1.2,1.3,1.4,1.6]
  scaled_incomes = list(map(lambda factor: int(round(income_series[num_family - 1] * factor, 0)), scaling_factors))
  
  for i, standard in enumerate(scaled_incomes):
    if standard >= income_avg:
      income_level = i+1
      break
    elif scaled_incomes[-1] < income_avg:
      income_level = len(scaling_factors)
      break
  return income_level


  ## 모든 조건 고려해 신청 유형 제안
  if marriage_status == 0: ## 신혼부부
    if income_status == 0: ## 맞벌이
      income_level 
      if child_status == 0: ## 신생아 양육(태아 포함)
          return {"type" : "신혼부부 특별공급",
                  "income_step" : (1,"신생아 우선공급")}
      

        elif 
        elif income_avg <= round(income_series[num_family-1]*1.3, 0):
          return {"type" : "생애최초 특별공급",
                  "income_step" : (1,"신생아 우선공급")}
        
    elif income_status == 1: ## 외벌이
      if income_avg <= income_series[num_family-1]:
        return {"type" : "신혼부부 특별공급",
                "income_step" : (1,"신생아 우선공급")}
    elif child_status == 1: ## 미성년 자녀 양육
  
  
